# Char2id

In [11]:
def file_padding(i):
    if i < 10: return '00000' + str(i)
    elif i < 100: return '0000' + str(i)
    elif i < 1000: return '000' + str(i)
    elif i < 10000: return '00' + str(i)
    elif i < 100000: return '0' + str(i)
    else: return str(i)

In [12]:
def get_path(path, fname, filenum, format_):
    return path+fname+filenum+format_

## Create kai_labels.csv

In [8]:
from tqdm import trange

path = "E:/한국어 음성데이터/KaiSpeech/"
fname = 'KaiSpeech_'
TOTAL_NUM = 622545
label_list = []
label_freq = []
print('started...')
for filenum in trange(1,TOTAL_NUM):
    f = open(get_path(path,fname,file_padding(filenum),".txt"))
    sentence = f.readline()
    f.close()
    for ch in sentence:
        if ch not in label_list:
            label_list.append(ch)
            label_freq.append(1)
        else:
            label_freq[label_list.index(ch)] += 1
label_freq, label_list = zip(*sorted(zip(label_freq, label_list), reverse=True))
label = {'id':[], 'char':[], 'freq' :[]}
for idx, ch_freq in enumerate(zip(label_list, label_freq)):
    ch, freq = ch_freq
    label['id'].append(idx)
    label['char'].append(ch)
    label['freq'].append(freq)

started...


100%|████████████████████████████████████████████████████████████████████████| 622544/622544 [03:02<00:00, 3409.03it/s]


NameError: name 'pd' is not defined

## Create KaiSpeech_label_FileNum

In [17]:
import pandas as pd
char2id = dict() # [ch] = id
id2char = dict() # [id] = ch

df_labels = pd.read_csv('kai_labels.csv', encoding='cp949')
id_list = df_labels['id']
ch_list = df_labels['char']

for idx, ch in enumerate(ch_list):
    char2id[ch] = id_list[idx]
    id2char[id_list[idx]] = ch

In [ ]:
from tqdm import trange

path = "E:/한국어 음성데이터/KaiSpeech/"
fname = 'KaiSpeech_'
label_fname = 'KaiSpeech_label_'
TOTAL_NUM = 622545

print('started...')
for filenum in trange(1,TOTAL_NUM):
    f = open(get_path(path,fname,file_padding(filenum),".txt"))
    sentence = f.readline()
    labels = str()
    f.close()
    for idx, ch in enumerate(sentence):
        if idx == 0: labels += str(char2id[ch])
        else : labels += (' ' + str(char2id[ch]))
    f = open(get_path(path,label_fname,file_padding(filenum),".txt"),"w")
    f.write(labels)
    f.close()
print("Ended!!!")

started...


  7%|████▋                                                                | 42307/622544 [1:26:59<233:35:06,  1.45s/it]

In [24]:
def label_to_string(labels, id2char):
    id_list = labels.split(' ')
    sentence = str()
    for id in id_list: sentence += id2char[int(id)]
    return sentence